# World happiness data across years

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc

### Dataframe explorations

In [ ]:
df_05_to_20 = pd.read_csv('datafiles/world-happiness-report-2005-2020.csv')

In [ ]:
df_05_to_20

In [ ]:
len(df_05_to_20['Country name'].unique())

In [ ]:
df_2021 = pd.read_csv('datafiles/world-happiness-report-2021.csv')

In [ ]:
df_2022 = pd.read_csv('datafiles/World Happiness Report 2022.csv')

In [ ]:
df_2023 = pd.read_csv('datafiles/WHR2023.csv')

In [ ]:
df_05_to_20_columns = df_05_to_20.columns
df_2021_columns = df_2021.columns
df_2022_columns = df_2022.columns
df_2023_columns = df_2023.columns

In [ ]:
df_05_to_20_columns

In [ ]:
df_2021_columns

### Default dataframe: 2005 - 2022 with some nulls

In [ ]:
df_05_22 = pd.read_csv('datafiles/regional20052022.csv')

In [ ]:
len(df_05_22['Country Name'].unique())

In [ ]:
list(df_05_22['Year'].unique())

### Default dataframe + 2023 (although lacking some data)

In [ ]:
df_2023.columns

In [ ]:
df_05_22.columns

In [ ]:
df_05_22_renamed = df_05_22.rename(columns={
    'Country Name' : 'Country name',
    'Life Ladder' : 'Ladder score',
    'Log GDP Per Capita': 'Logged GDP per capita',
    'Social Support': 'Social support',
    'Healthy Life Expectancy At Birth': 'Healthy life expectancy',
    'Freedom To Make Life Choices': 'Freedom to make life choices',
    'Perceptions Of Corruption': 'Perceptions of corruption',
})

In [ ]:
unified_columns = ['Country name', 'Ladder score', 'Logged GDP per capita',
                  'Social support', 'Healthy life expectancy',
                  'Freedom to make life choices', 'Perceptions of corruption', 'Year',
                  'Regional Indicator']

In [ ]:
combined_2023_df = pd.read_csv('datafiles/combined_df_2023.csv', index_col=0)

In [ ]:
combined_2023_df.columns

In [ ]:
combined_2023_df['Year'] = 2023

In [ ]:
df_2023_renamed = combined_2023_df.rename(columns={
    'Country' : 'Country name',
    'Life expectancy': 'Healthy life expectancy',
    'Freedom': 'Freedom to make life choices',
    'Corruption perception': 'Perceptions of corruption',
})

In [ ]:
df_05_22_uni = df_05_22_renamed[[*unified_columns]]

In [ ]:
df_23_uni = df_2023_renamed[[*unified_columns]]

In [ ]:
df_05_23 = pd.concat([df_05_22_uni, df_23_uni], axis=0)

In [ ]:
df_05_23

In [ ]:
df_05_23['Year'].unique()

In [ ]:
df_05_23.to_csv('datafiles/world-05-23-unified.csv')

### Up to 2022 explorations
df_05_22

In [ ]:
df_05_22.columns

In [ ]:
df_05_22.sample(5)

In [ ]:
gdp_year_fig = go.Figure()
gdp_year_fig.add_trace(
    go.Line(
        x=df_05_22.query("`Country Name` == 'Finland'")['Year'],
        y=df_05_22.query("`Country Name` == 'Finland'")['Log GDP Per Capita'],
        name='Finland'
    )
)

gdp_year_fig.add_trace(
    go.Line(
        x=df_05_22.query("`Country Name` == 'Bolivia'")['Year'],
        y=df_05_22.query("`Country Name` == 'Bolivia'")['Log GDP Per Capita'],
        name='Bolivia'
    )
)

In [ ]:
life_year_fig = go.Figure()
life_year_fig.add_trace(
    go.Line(
        x=df_05_22.query("`Country Name` == 'New Zealand'")['Year'],
        y=df_05_22.query("`Country Name` == 'New Zealand'")['Healthy Life Expectancy At Birth'],
        name='New Zealand'
    )
)

life_year_fig.add_trace(
    go.Line(
        x=df_05_22.query("`Country Name` == 'Saudi Arabia'")['Year'],
        y=df_05_22.query("`Country Name` == 'Saudi Arabia'")['Healthy Life Expectancy At Birth'],
        name='Saudi Arabia'
    )
)

## App

In [ ]:
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

### Metric per year for country in region

In [ ]:
regions = list(df_05_22['Regional Indicator'].dropna().unique())

In [ ]:
regions

In [ ]:
metrics = [
    'Life Ladder', 'Log GDP Per Capita', 'Social Support',
    'Healthy Life Expectancy At Birth', 'Freedom To Make Life Choices',
    'Generosity', 'Perceptions Of Corruption', 'Positive Affect',
    'Negative Affect', 'Confidence In National Government'
]

#### choose metric, region and countries and display for each year 

In [ ]:
metric_year_country_controls = dbc.Card(
    [
        html.Div(
            [
            dbc.Label("Region"),
            dcc.Dropdown(
                id='metric-year-region-selection',
                value = ['Western Europe',],
                options = regions,
                multi = True
            )
            ]),
        html.Div(
            [
            dbc.Label("Country"),
            dcc.Dropdown(
                id='metric-year-country-selection',
                multi=True
            )
            ]),
        html.Div(
            [
            dbc.Label("Metric"),
            dcc.Dropdown(
                id='metric-year-metric-selection',
                value = 'Life Ladder',
                options = metrics
            )
            ]),
        # html.Div(
        #     [
        #     dbc.Label("Year"),
        #     dcc.Slider(
        #         2005, 2022, 1,
        #         value = 2005,
        #         id='metric-year-year-selection',
        #     )
        #     ]),
    ],
    body=True,
)

In [ ]:
metric_year_country_container = dbc.Container([
    html.H1(children = 'Metric for chosen year and country within region', style={'textAlign': 'center'}),
    html.P(id='metric-year-country-shape'),
    dbc.Row(
        [
            dbc.Col(metric_year_country_controls, md=4),
            dbc.Col(dcc.Graph(id='metric-year-country-graph'), md=8),
        ],
        align="center",
    ),
], fluid=True)

In [ ]:
@app.callback(
    [
        Output('metric-year-country-graph', 'figure'),
        Output('metric-year-country-shape', 'children'),
    ],
    [
        Input('metric-year-country-selection', 'value'),
        Input('metric-year-metric-selection', 'value'),
        # Input('metric-year-year-selection', 'value'),
    ]
)
def update_metric_year_country_graph(countries, metric):
    fig = go.Figure()

    for country in countries:
        country_df = df_05_22[df_05_22['Country Name'] == country]
        fig.add_trace(
            go.Line(x = country_df['Year'].values,
                    y = country_df[metric].values,
                    name = country
                   )
        )
    # fig.update_layout(barmode='relative')
    fig.update_layout(barmode='stack')
    
    # return fig, f'shape: {country_df.shape}'
    return fig, f'countries: {countries}'

@app.callback(
    [
        Output('metric-year-country-selection', 'options')
    ],
    [
        Input('metric-year-region-selection', 'value')
    ]
)
def set_dynamic_metric_year_country_options(regions):
    options = dict()
    for region in regions:
        region_countries = list(df_05_22[df_05_22['Regional Indicator'] == region]['Country Name'].unique())
    for country in region_countries:
        options[country] = country
    return [options]

# ======= choose from available dynamic options =========
# has to be done manually due to dynamicity
# also this is how to return multiple options
# in case of one you'd have to take [0] of value from options
@app.callback(
    Output('metric-year-country-selection', 'value'),
    Input('metric-year-country-selection', 'options')
)
def set_dynamic_metric_year_country_value(options):
    output = [option for option in options]
    return output
    # return options[0]['value']
    # return options

#### Choose year and metric and display all regions

In [ ]:
metric_regions_controls = dbc.Card(
    [
        html.Div(
            [
            dbc.Label("Metric"),
            dcc.Dropdown(
                id='metric-regions-metric-selection',
                value = 'Life Ladder',
                options = metrics
            )
            ]),
        html.Div(
            [
            dbc.Label("Year"),
            dcc.Slider(
                2005, 2022, 1,
                value = 2012,
                id='metric-regions-year-selection',
            )
            ]),
    ],
    body=True,
)

In [ ]:
metric_regions_container = dbc.Container([
    html.H1(children = 'Metric by regions for selected year', style={'textAlign': 'center'}),
    html.P(id='metric-regions-test'),
    dbc.Row(
        [
            dbc.Col(metric_regions_controls, md=4),
            dbc.Col(dcc.Graph(id='metric-regions-graph'), md=8),
        ],
        align="center",
    ),
], fluid=True)

In [ ]:
@app.callback(
    [
        Output('metric-regions-graph', 'figure')
        ,Output('metric-regions-test', 'children')
    ],
    [
        Input('metric-regions-metric-selection', 'value'),
        Input('metric-regions-year-selection', 'value'),
    ]
)
def update_metric_regions_graph(metric, year):
    year_df = df_05_22[df_05_22['Year'] == year]
    fig = px.bar(year_df, x = 'Regional Indicator', y = metric, barmode='stack') 
    # fig = px.scatter(df, y='Life expectancy', x=metric, hover_data='Country', color='Regional Indicator')
    return fig, f'metric: {metric}'
    # return fig

### App layout and run

In [ ]:
app.layout = dbc.Container([
    metric_year_country_container,
    html.Hr(),
    metric_regions_container,
    html.Hr()
], fluid=True)

In [ ]:
app.run(debug=True, port=8051)
# app.run(port=8051)

### Some manual visualizations

In [ ]:
metrics_means = df_05_22.groupby(by=['Regional Indicator', 'Year'], as_index=False)[metrics].mean()

In [ ]:
metrics_means

In [ ]:
# metrics_means[metrics_means['Regional Indicator'] == 'Western Europe']

In [ ]:
means_fig = go.Figure()

for region in regions:
    region_df = metrics_means[metrics_means['Regional Indicator'] == region]
    means_fig.add_trace(
        go.Line(x=region_df['Year'], y=region_df['Life Ladder'], name=region)
    )

In [ ]:
means_fig.update_layout(
    title = "Ladder score over years regional means",
    xaxis_title="Year",
    yaxis_title="Life Ladder",
    width = 800,
    height = 600,
    legend_title = "Regions"
)
means_fig.show()

### Animations

In [ ]:
metrics

In [ ]:
px.scatter(metrics_means, x='Log GDP Per Capita', y='Healthy Life Expectancy At Birth',
          size='Life Ladder', animation_frame='Year', color='Regional Indicator',
          height=600, width=800)